# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [12]:
import os
from glob import glob

# Write your code below.
parquet_files = glob(os.path.join(os.getenv('PRICE_DATA'), "**/*.parquet"),recursive=True)

print(len(parquet_files))


2906


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [18]:
# Write your code below.
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_feat = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
)
dd_feat = dd_feat.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)

dd_feat = dd_feat.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Adj_Close_lag_1 = x['Adj Close'].shift(1))
)

dd_feat = dd_feat.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(hi_lo_range = x['Close'].max() - x['Close'].min())
)



/var/folders/z3/ts01c4l53qz5tch9p89ynqfc0000gn/T/ipykernel_23637/1203659529.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_px.groupby('ticker', group_keys=False).apply(
/var/folders/z3/ts01c4l53qz5tch9p89ynqfc0000gn/T/ipykernel_23637/1203659529.py:10: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_feat.groupby('ticker', group_keys=False).apply(
/var/folders/z3/ts01c4l53qz5tch9p89ynqfc0000gn/T/ipykernel_23637/1203659529.py:14: UserWarning: `meta` is not specified, inferred from partial da

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [28]:
# Write your code below.
dd_feat = dd_feat.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(rolling_returns = x['Returns'].rolling(10).mean())
)
dd_feat.compute()


/var/folders/z3/ts01c4l53qz5tch9p89ynqfc0000gn/T/ipykernel_23637/564120533.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_feat.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Returns,Adj_Close_lag_1,hi_lo_range,rolling_returns
ticker,,,,,,,,,,,,,,
TERP,2016-06-09,8.95,8.950000,8.530000,8.680000,6.322244,1749600.0,TERP.csv,2016,9.09,-0.045104,6.620875,35.250001,NaN
TERP,2017-02-22,11.67,11.790000,11.530000,11.560000,8.419948,494300.0,TERP.csv,2017,11.71,-0.012810,8.529202,35.250001,NaN
TERP,2017-01-18,12.64,12.710000,12.340000,12.420000,9.046344,325600.0,TERP.csv,2017,12.56,-0.011147,9.148318,35.250001,NaN
TERP,2018-12-04,11.62,11.760000,11.500000,11.550000,10.804018,706200.0,TERP.csv,2018,11.65,-0.008584,10.897559,35.250001,NaN
TERP,2018-06-06,10.85,10.850000,10.750000,10.780000,9.749771,460900.0,TERP.csv,2018,10.82,-0.003697,9.785947,35.250001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IPDN,2015-05-20,21.92,22.799999,21.280001,22.799999,22.799999,3400.0,IPDN.csv,2015,21.52,0.059479,21.520000,60.490001,0.001657
IPDN,2017-05-18,8.90,9.950000,8.800000,9.500000,9.500000,127900.0,IPDN.csv,2017,8.75,0.085714,8.750000,60.490001,0.013570
IPDN,2016-12-19,8.36,9.080000,8.300000,9.080000,9.080000,7000.0,IPDN.csv,2016,8.36,0.086124,8.360000,60.490001,0.024374


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Each partition in a Dask DataFrame is a Pandas DataFrame. Running df.compute() will coalesce all the underlying partitions in the Dask DataFrame into a single Pandas DataFrame. That'll cause problems if the size of the Pandas DataFrame is bigger than the RAM on your machine.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.